In [1]:

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate
from sklearn.compose import make_column_selector as selector
from sklearn.impute import MissingIndicator
from sklearn.impute import SimpleImputer


In [2]:
data = pd.read_csv('../data/LISS_example_input_data.csv', encoding='cp1252')
outcome = pd.read_csv('../data/LISS_example_groundtruth_data.csv')


/tmp/ipykernel_7167/2595860588.py:1: DtypeWarning: Columns (209,210,369,2824,2825,2826,2827,2828,2829,2830,4478,4479,4480,4481,4483,4484,4485,4486,4490,4491,4492,4493,4495,4496,4497,4498,4502,4503,4504,4505,4507,4508,4509,4510,4514,4515,4516,4517,4519,4520,4521,4522,5229,5231,5566,5567,5569,5570,5572,5573,5575,5576,5578,5579,5581,5582,5584,5585,5587,5588,5721,5722,5723,5724,5725,5726,5727,5728,5729,5730,6289,6290,6291,6292,6293,6294,6295,6296,6297,6298,6299,6502,6503,6504,6505,6506,6507,6508,6509,6510,6511,6512,6513,6514,6515,6516,6517,6518,6519,6520,6521,6522,6523,6524,6525,6526,6527,6528,6529,6530,6531,6532,6533,6534,6535,6536,6537,6538,6539,6540,6541,6542,6543,6544,6545,6546,6547,6548,6549,6822,6823,6824,6825,6826,6827,6828,6829,6830,6831,6832,6921,6922,6923,6924,6925,6926,6927,6928,6929,6930,6931,7020,7021,7022,7023,7024,7026,7027,7028,7029,7030,7247,7248,7249,7250,7251,7252,7253,7254,7255,7256,7257,7258,8586,8587,8588,8589,8590,8591,8592,8593,8594,8595,8596,9703,9704,9705,9706,970

In [3]:
data_c = data.copy()

In [4]:

# Defining an household id and dropping those which are nas.
data = data.rename(columns = {'nohouse_encr2019':'hh_id'})
# Identify columns to be dropped
columns_to_drop = [col for col in data.columns if col.startswith('nohouse_encr')]

# Drop the columns
data = data.drop(columns_to_drop, axis=1)

# Filtering out those households that do not have an household id.
data = data.loc[data['hh_id'].notna()]
outcome = outcome.merge(data[['nomem_encr','hh_id']], on = 'nomem_encr')

# Filtering out individuals where target (new_child) is na.
outcome = outcome.loc[outcome['new_child'].notna()]
data = data.loc[data['hh_id'].isin(outcome['hh_id'])]

# Dropping columns with only na values.
data = data.loc[:,data.isna().sum() != len(data)].copy()


outcome = pd.concat([outcome,data[['nomem_encr','hh_id']].assign(new_child = 0)], ignore_index=True)
outcome = outcome.sort_values(by = 'new_child', ascending = False)
outcome = outcome.drop_duplicates(subset = 'nomem_encr')
outcome = outcome.set_index(['hh_id','nomem_encr'])


# Setting the index.
data = data.set_index(['hh_id','nomem_encr'])


data = data.loc[:,['leeftijd2019',
          'geslacht',
          'positie2019',
          'aantalhh2019',
          'aantalki2019',
          'partner2019',
          'burgstat2019',
          'woonvorm2019',
          'woning2019',
          'sted2019',
          'belbezig2019',
          'brutoink2019',
          'brutoink_f2019',
          'netinc2019',
          'brutohh_f2019',
          'nettohh_f2019',
          'oplzon2019',
          'oplmet2019',
          'doetmee2019',
          'herkomstgroep2019',
          'simpc2019']]


In [ ]:


# Impute missing values, and add features that flag missing values.
imputer = SimpleImputer(missing_values=np.nan, strategy='mean', fill_value=True, add_indicator=True)

# BE CAREFUL WITH THE FLAGS FOR MISSING VALUES IN THE COLUMN SELECTION.
# Selecting categorical columns.
categorical_columns_selector = selector(dtype_include = object)
# Getting the names of the categorical columns.
categorical_columns = categorical_columns_selector(data)
# Encoding the categorical columns.
encoder = OneHotEncoder(handle_unknown='ignore')

# Selecting numerical columns.
numerical_columns_selector = selector(dtype_include = "float64")
# Getting the name of the numerical columns.
numerical_columns = numerical_columns_selector(data)
# Scaling numerical columns.
scaler = StandardScaler()

# Probably not needed.
# # Create the object flagging missing values.
# missing_values = MissingIndicator()
# # Flagging na values.
# #flags = missing_values.fit_transform(data)
# # Get the feature names for the flags (columns spelled 'missing' + column name).
# names_columns_missing = missing_values.get_feature_names_out() 

preprocessor = ColumnTransformer(
    [
        ('on-hot-encoder',encoder,categorical_columns),
        ('standard_scaler',scaler,numerical_columns)
    ]
)

model = make_pipeline(imputer,preprocessor,LogisticRegression(max_iter=500))
# replace income by income categories if not available?

#cross_validate(model,data,outcome,cv = 5)


